# Introduction to Mortgage ETL Job
This is the mortgage ETL job to generate the input datasets for the mortgage Xgboost job.


## Prerequirement
### 1. Download data
Dataset is downloaded from [Fannie Mae’s Single-Family Loan Performance Data](https://capitalmarkets.fanniemae.com/credit-risk-transfer/single-family-credit-risk-transfer/fannie-mae-single-family-loan-performance-data) with all rights reserved by Fannie Mae.

### 2. Download needed jars
* [cudf-22.04.0-cuda11.jar](https://repo1.maven.org/maven2/ai/rapids/cudf/22.04.0/)
* [rapids-4-spark_2.12-22.04.0.jar](https://repo1.maven.org/maven2/com/nvidia/rapids-4-spark_2.12/22.04.0/rapids-4-spark_2.12-22.04.0.jar)

### 3. Start Spark Standalone
Before Running the script, please setup Spark standalone mode

### 4. Add ENV
```
$ export SPARK_JARS=cudf-22.04.0-cuda11.jar,rapids-4-spark_2.12-22.04.0.jar

```

### 5.Start Jupyter Notebook with spylon-kernal or toree

```
$ jupyter notebook --allow-root --notebook-dir=${your-dir} --config=${your-configs}
```

## Import Libs

In [1]:
import org.apache.hadoop.fs.Path
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql.{Column, DataFrame, SparkSession}

Intitializing Scala interpreter ...

Spark Web UI available at http://hazhu-dt:4040
SparkContext available as 'sc' (version = 3.2.1, master = spark://hazhu-dt:7077, app id = app-20220526191430-0014)
SparkSession available as 'spark'


import org.apache.hadoop.fs.Path
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql.{Column, DataFrame, SparkSession}


In [33]:
spark.conf.set("spark.rapids.sql.csv.read.double.enabled",true)
spark.conf.set("spark.rapids.sql.enabled",true)
spark.conf.set("spark.sql.adaptive.enabled",true)

## Script Settings

### 1. File Path Settings
* Define input file path

In [34]:
val dataRoot = sys.env.getOrElse("DATA_ROOT", "/home/hazhu/data/mortgage_raw")
val rawPath = dataRoot + "/input/"
val outPath = dataRoot + "/output/"


dataRoot: String = /home/hazhu/data/mortgage_raw
rawPath: String = /home/hazhu/data/mortgage_raw/input/
outPath: String = /home/hazhu/data/mortgage_raw/output/


## Function and Object Define
### 1. Define the constants

* Define input/output file schema (Performance and Acquisition)

In [35]:
// File schema
val performanceSchema = StructType(Array(
      StructField("loan_id", LongType),
      StructField("monthly_reporting_period", StringType),
      StructField("servicer", StringType),
      StructField("interest_rate", DoubleType),
      StructField("current_actual_upb", DoubleType),
      StructField("loan_age", DoubleType),
      StructField("remaining_months_to_legal_maturity", DoubleType),
      StructField("adj_remaining_months_to_maturity", DoubleType),
      StructField("maturity_date", StringType),
      StructField("msa", DoubleType),
      StructField("current_loan_delinquency_status", IntegerType),
      StructField("mod_flag", StringType),
      StructField("zero_balance_code", StringType),
      StructField("zero_balance_effective_date", StringType),
      StructField("last_paid_installment_date", StringType),
      StructField("foreclosed_after", StringType),
      StructField("disposition_date", StringType),
      StructField("foreclosure_costs", DoubleType),
      StructField("prop_preservation_and_repair_costs", DoubleType),
      StructField("asset_recovery_costs", DoubleType),
      StructField("misc_holding_expenses", DoubleType),
      StructField("holding_taxes", DoubleType),
      StructField("net_sale_proceeds", DoubleType),
      StructField("credit_enhancement_proceeds", DoubleType),
      StructField("repurchase_make_whole_proceeds", StringType),
      StructField("other_foreclosure_proceeds", DoubleType),
      StructField("non_interest_bearing_upb", DoubleType),
      StructField("principal_forgiveness_upb", StringType),
      StructField("repurchase_make_whole_proceeds_flag", StringType),
      StructField("foreclosure_principal_write_off_amount", StringType),
      StructField("servicing_activity_indicator", StringType))
    )

val acquisitionSchema = StructType(Array(
      StructField("loan_id", LongType),
      StructField("orig_channel", StringType),
      StructField("seller_name", StringType),
      StructField("orig_interest_rate", DoubleType),
      StructField("orig_upb", IntegerType),
      StructField("orig_loan_term", IntegerType),
      StructField("orig_date", StringType),
      StructField("first_pay_date", StringType),
      StructField("orig_ltv", DoubleType),
      StructField("orig_cltv", DoubleType),
      StructField("num_borrowers", DoubleType),
      StructField("dti", DoubleType),
      StructField("borrower_credit_score", DoubleType),
      StructField("first_home_buyer", StringType),
      StructField("loan_purpose", StringType),
      StructField("property_type", StringType),
      StructField("num_units", IntegerType),
      StructField("occupancy_status", StringType),
      StructField("property_state", StringType),
      StructField("zip", IntegerType),
      StructField("mortgage_insurance_percent", DoubleType),
      StructField("product_type", StringType),
      StructField("coborrow_credit_score", DoubleType),
      StructField("mortgage_insurance_type", DoubleType),
      StructField("relocation_mortgage_indicator", StringType))
    )

performanceSchema: org.apache.spark.sql.types.StructType = StructType(StructField(loan_id,LongType,true), StructField(monthly_reporting_period,StringType,true), StructField(servicer,StringType,true), StructField(interest_rate,DoubleType,true), StructField(current_actual_upb,DoubleType,true), StructField(loan_age,DoubleType,true), StructField(remaining_months_to_legal_maturity,DoubleType,true), StructField(adj_remaining_months_to_maturity,DoubleType,true), StructField(maturity_date,StringType,true), StructField(msa,DoubleType,true), StructField(current_loan_delinquency_status,IntegerType,true), StructField(mod_flag,StringType,true), StructField(zero_balance_code,StringType,true), StructField(zero_balance_effective_date,StringType,true), StructField(last_paid_installment_date,StringType,t...


* Define seller name mapping

In [36]:
object NameMapping {
  /**
    * Returns a dataframe with two columns named based off of the column names passed in.
    * The fromColName has the original name we want to clean up, the toColName
    * will have the name we want to go to, the unambiguous name.
    */
  def apply(spark: SparkSession, fromColName: String, toColName: String): DataFrame = {
    import spark.sqlContext.implicits._
    broadcast(Seq(
      ("WITMER FUNDING, LLC", "Witmer"),
      ("WELLS FARGO CREDIT RISK TRANSFER SECURITIES TRUST 2015", "Wells Fargo"),
      ("WELLS FARGO BANK,  NA" , "Wells Fargo"),
      ("WELLS FARGO BANK, N.A." , "Wells Fargo"),
      ("WELLS FARGO BANK, NA" , "Wells Fargo"),
      ("USAA FEDERAL SAVINGS BANK" , "USAA"),
      ("UNITED SHORE FINANCIAL SERVICES, LLC D\\/B\\/A UNITED WHOLESALE MORTGAGE" , "United Seq(e"),
      ("U.S. BANK N.A." , "US Bank"),
      ("SUNTRUST MORTGAGE INC." , "Suntrust"),
      ("STONEGATE MORTGAGE CORPORATION" , "Stonegate Mortgage"),
      ("STEARNS LENDING, LLC" , "Stearns Lending"),
      ("STEARNS LENDING, INC." , "Stearns Lending"),
      ("SIERRA PACIFIC MORTGAGE COMPANY, INC." , "Sierra Pacific Mortgage"),
      ("REGIONS BANK" , "Regions"),
      ("RBC MORTGAGE COMPANY" , "RBC"),
      ("QUICKEN LOANS INC." , "Quicken Loans"),
      ("PULTE MORTGAGE, L.L.C." , "Pulte Mortgage"),
      ("PROVIDENT FUNDING ASSOCIATES, L.P." , "Provident Funding"),
      ("PROSPECT MORTGAGE, LLC" , "Prospect Mortgage"),
      ("PRINCIPAL RESIDENTIAL MORTGAGE CAPITAL RESOURCES, LLC" , "Principal Residential"),
      ("PNC BANK, N.A." , "PNC"),
      ("PMT CREDIT RISK TRANSFER TRUST 2015-2" , "PennyMac"),
      ("PHH MORTGAGE CORPORATION" , "PHH Mortgage"),
      ("PENNYMAC CORP." , "PennyMac"),
      ("PACIFIC UNION FINANCIAL, LLC" , "Other"),
      ("OTHER" , "Other"),
      ("NYCB MORTGAGE COMPANY, LLC" , "NYCB"),
      ("NEW YORK COMMUNITY BANK" , "NYCB"),
      ("NETBANK FUNDING SERVICES" , "Netbank"),
      ("NATIONSTAR MORTGAGE, LLC" , "Nationstar Mortgage"),
      ("METLIFE BANK, NA" , "Metlife"),
      ("LOANDEPOT.COM, LLC" , "LoanDepot.com"),
      ("J.P. MORGAN MADISON AVENUE SECURITIES TRUST, SERIES 2015-1" , "JP Morgan Chase"),
      ("J.P. MORGAN MADISON AVENUE SECURITIES TRUST, SERIES 2014-1" , "JP Morgan Chase"),
      ("JPMORGAN CHASE BANK, NATIONAL ASSOCIATION" , "JP Morgan Chase"),
      ("JPMORGAN CHASE BANK, NA" , "JP Morgan Chase"),
      ("JP MORGAN CHASE BANK, NA" , "JP Morgan Chase"),
      ("IRWIN MORTGAGE, CORPORATION" , "Irwin Mortgage"),
      ("IMPAC MORTGAGE CORP." , "Impac Mortgage"),
      ("HSBC BANK USA, NATIONAL ASSOCIATION" , "HSBC"),
      ("HOMEWARD RESIDENTIAL, INC." , "Homeward Mortgage"),
      ("HOMESTREET BANK" , "Other"),
      ("HOMEBRIDGE FINANCIAL SERVICES, INC." , "HomeBridge"),
      ("HARWOOD STREET FUNDING I, LLC" , "Harwood Mortgage"),
      ("GUILD MORTGAGE COMPANY" , "Guild Mortgage"),
      ("GMAC MORTGAGE, LLC (USAA FEDERAL SAVINGS BANK)" , "GMAC"),
      ("GMAC MORTGAGE, LLC" , "GMAC"),
      ("GMAC (USAA)" , "GMAC"),
      ("FREMONT BANK" , "Fremont Bank"),
      ("FREEDOM MORTGAGE CORP." , "Freedom Mortgage"),
      ("FRANKLIN AMERICAN MORTGAGE COMPANY" , "Franklin America"),
      ("FLEET NATIONAL BANK" , "Fleet National"),
      ("FLAGSTAR CAPITAL MARKETS CORPORATION" , "Flagstar Bank"),
      ("FLAGSTAR BANK, FSB" , "Flagstar Bank"),
      ("FIRST TENNESSEE BANK NATIONAL ASSOCIATION" , "Other"),
      ("FIFTH THIRD BANK" , "Fifth Third Bank"),
      ("FEDERAL HOME LOAN BANK OF CHICAGO" , "Fedral Home of Chicago"),
      ("FDIC, RECEIVER, INDYMAC FEDERAL BANK FSB" , "FDIC"),
      ("DOWNEY SAVINGS AND LOAN ASSOCIATION, F.A." , "Downey Mortgage"),
      ("DITECH FINANCIAL LLC" , "Ditech"),
      ("CITIMORTGAGE, INC." , "Citi"),
      ("CHICAGO MORTGAGE SOLUTIONS DBA INTERFIRST MORTGAGE COMPANY" , "Chicago Mortgage"),
      ("CHICAGO MORTGAGE SOLUTIONS DBA INTERBANK MORTGAGE COMPANY" , "Chicago Mortgage"),
      ("CHASE HOME FINANCE, LLC" , "JP Morgan Chase"),
      ("CHASE HOME FINANCE FRANKLIN AMERICAN MORTGAGE COMPANY" , "JP Morgan Chase"),
      ("CHASE HOME FINANCE (CIE 1)" , "JP Morgan Chase"),
      ("CHASE HOME FINANCE" , "JP Morgan Chase"),
      ("CASHCALL, INC." , "CashCall"),
      ("CAPITAL ONE, NATIONAL ASSOCIATION" , "Capital One"),
      ("CALIBER HOME LOANS, INC." , "Caliber Funding"),
      ("BISHOPS GATE RESIDENTIAL MORTGAGE TRUST" , "Bishops Gate Mortgage"),
      ("BANK OF AMERICA, N.A." , "Bank of America"),
      ("AMTRUST BANK" , "AmTrust"),
      ("AMERISAVE MORTGAGE CORPORATION" , "Amerisave"),
      ("AMERIHOME MORTGAGE COMPANY, LLC" , "AmeriHome Mortgage"),
      ("ALLY BANK" , "Ally Bank"),
      ("ACADEMY MORTGAGE CORPORATION" , "Academy Mortgage"),
      ("NO CASH-OUT REFINANCE" , "OTHER REFINANCE"),
      ("REFINANCE - NOT SPECIFIED" , "OTHER REFINANCE"),
      ("Other REFINANCE" , "OTHER REFINANCE")
    ).toDF(fromColName, toColName))
  }
}

defined object NameMapping


In [37]:
val rawSchema = StructType(Array(
      StructField("reference_pool_id", StringType),
      StructField("loan_id", LongType),
      StructField("monthly_reporting_period", StringType),
      StructField("orig_channel", StringType),
      StructField("seller_name", StringType),
      StructField("servicer", StringType),
      StructField("master_servicer", StringType),
      StructField("orig_interest_rate", DoubleType),
      StructField("interest_rate", DoubleType),
      StructField("orig_upb", IntegerType),
      StructField("upb_at_issuance", StringType),
      StructField("current_actual_upb", DoubleType),
      StructField("orig_loan_term", IntegerType),
      StructField("orig_date", StringType),
      StructField("first_pay_date", StringType),    
      StructField("loan_age", DoubleType),
      StructField("remaining_months_to_legal_maturity", DoubleType),
      StructField("adj_remaining_months_to_maturity", DoubleType),
      StructField("maturity_date", StringType),
      StructField("orig_ltv", DoubleType),
      StructField("orig_cltv", DoubleType),
      StructField("num_borrowers", DoubleType),
      StructField("dti", DoubleType),
      StructField("borrower_credit_score", DoubleType),
      StructField("coborrow_credit_score", DoubleType),
      StructField("first_home_buyer", StringType),
      StructField("loan_purpose", StringType),
      StructField("property_type", StringType),
      StructField("num_units", IntegerType),
      StructField("occupancy_status", StringType),
      StructField("property_state", StringType),
      StructField("msa", DoubleType),
      StructField("zip", IntegerType),
      StructField("mortgage_insurance_percent", DoubleType),
      StructField("product_type", StringType),
      StructField("prepayment_penalty_indicator", StringType),
      StructField("interest_only_loan_indicator", StringType),
      StructField("interest_only_first_principal_and_interest_payment_date", StringType),
      StructField("months_to_amortization", StringType),
      StructField("current_loan_delinquency_status", IntegerType),
      StructField("loan_payment_history", StringType),
      StructField("mod_flag", StringType),
      StructField("mortgage_insurance_cancellation_indicator", StringType),
      StructField("zero_balance_code", StringType),
      StructField("zero_balance_effective_date", StringType),
      StructField("upb_at_the_time_of_removal", StringType),
      StructField("repurchase_date", StringType),
      StructField("scheduled_principal_current", StringType),
      StructField("total_principal_current", StringType),
      StructField("unscheduled_principal_current", StringType),
      StructField("last_paid_installment_date", StringType),
      StructField("foreclosed_after", StringType),
      StructField("disposition_date", StringType),
      StructField("foreclosure_costs", DoubleType),
      StructField("prop_preservation_and_repair_costs", DoubleType),
      StructField("asset_recovery_costs", DoubleType),
      StructField("misc_holding_expenses", DoubleType),
      StructField("holding_taxes", DoubleType),
      StructField("net_sale_proceeds", DoubleType),
      StructField("credit_enhancement_proceeds", DoubleType),
      StructField("repurchase_make_whole_proceeds", StringType),
      StructField("other_foreclosure_proceeds", DoubleType),
      StructField("non_interest_bearing_upb", DoubleType),
      StructField("principal_forgiveness_upb", StringType),
      StructField("original_list_start_date", StringType),
      StructField("original_list_price", StringType),
      StructField("current_list_start_date", StringType),
      StructField("current_list_price", StringType),
      StructField("borrower_credit_score_at_issuance", StringType),
      StructField("co-borrower_credit_score_at_issuance", StringType),
      StructField("borrower_credit_score_current", StringType),
      StructField("co-Borrower_credit_score_current", StringType),
      StructField("mortgage_insurance_type", DoubleType),
      StructField("servicing_activity_indicator", StringType),
      StructField("current_period_modification_loss_amount", StringType),
      StructField("cumulative_modification_loss_amount", StringType),
      StructField("current_period_credit_event_net_gain_or_loss", StringType),
      StructField("cumulative_credit_event_net_gain_or_loss", StringType),
      StructField("homeready_program_indicator", StringType),
      StructField("foreclosure_principal_write_off_amount", StringType),
      StructField("relocation_mortgage_indicator", StringType),
      StructField("zero_balance_code_change_date", StringType),
      StructField("loan_holdback_indicator", StringType),
      StructField("loan_holdback_effective_date", StringType),
      StructField("delinquent_accrued_interest", StringType),
      StructField("property_valuation_method", StringType),
      StructField("high_balance_loan_indicator", StringType),
      StructField("arm_initial_fixed-rate_period_lt_5_yr_indicator", StringType),
      StructField("arm_product_type", StringType),
      StructField("initial_fixed-rate_period", StringType),
      StructField("interest_rate_adjustment_frequency", StringType),
      StructField("next_interest_rate_adjustment_date", StringType),
      StructField("next_payment_change_date", StringType),
      StructField("index", StringType),
      StructField("arm_cap_structure", StringType),
      StructField("initial_interest_rate_cap_up_percent", StringType),
      StructField("periodic_interest_rate_cap_up_percent", StringType),
      StructField("lifetime_interest_rate_cap_up_percent", StringType),
      StructField("mortgage_margin", StringType),
      StructField("arm_balloon_indicator", StringType),
      StructField("arm_plan_number", StringType),
      StructField("borrower_assistance_plan", StringType),
      StructField("hltv_refinance_option_indicator", StringType),
      StructField("deal_name", StringType),
      StructField("repurchase_make_whole_proceeds_flag", StringType),
      StructField("alternative_delinquency_resolution", StringType),
      StructField("alternative_delinquency_resolution_count", StringType),
      StructField("total_deferral_amount", StringType)
      )
    )

rawSchema: org.apache.spark.sql.types.StructType = StructType(StructField(reference_pool_id,StringType,true), StructField(loan_id,LongType,true), StructField(monthly_reporting_period,StringType,true), StructField(orig_channel,StringType,true), StructField(seller_name,StringType,true), StructField(servicer,StringType,true), StructField(master_servicer,StringType,true), StructField(orig_interest_rate,DoubleType,true), StructField(interest_rate,DoubleType,true), StructField(orig_upb,IntegerType,true), StructField(upb_at_issuance,StringType,true), StructField(current_actual_upb,DoubleType,true), StructField(orig_loan_term,IntegerType,true), StructField(orig_date,StringType,true), StructField(first_pay_date,StringType,true), StructField(loan_age,DoubleType,true), StructField(remaining_months...


### 2. Define ETL Process

Define the function to do the ETL process


* Define category (string) column and numeric column

In [38]:
val labelColName = "delinquency_12"
val categaryCols = List(
    ("orig_channel", FloatType),
    ("first_home_buyer", FloatType),
    ("loan_purpose", FloatType),
    ("property_type", FloatType),
    ("occupancy_status", FloatType),
    ("property_state", FloatType),
    ("product_type", FloatType),
    ("relocation_mortgage_indicator", FloatType),
    ("seller_name", FloatType),
    ("mod_flag", FloatType)
  )

val numericCols = List(
    ("orig_interest_rate", FloatType),
    ("orig_upb", IntegerType),
    ("orig_loan_term", IntegerType),
    ("orig_ltv", FloatType),
    ("orig_cltv", FloatType),
    ("num_borrowers", FloatType),
    ("dti", FloatType),
    ("borrower_credit_score", FloatType),
    ("num_units", IntegerType),
    ("zip", IntegerType),
    ("mortgage_insurance_percent", FloatType),
    ("current_loan_delinquency_status", IntegerType),
    ("current_actual_upb", FloatType),
    ("interest_rate", FloatType),
    ("loan_age", FloatType),
    ("msa", FloatType),
    ("non_interest_bearing_upb", FloatType),
    (labelColName, IntegerType)
  )

val commParamMap = Map(
    "eta" -> 0.1,
    "gamma" -> 0.1,
    "missing" -> 0.0,
    "max_depth" -> 10,
    "max_leaves" -> 256,
    "objective" -> "binary:logistic",
    "grow_policy" -> "depthwise",
    "min_child_weight" -> 30,
    "lambda" -> 1,
    "scale_pos_weight" -> 2,
    "subsample" -> 1,
    "nthread" -> 1,
    "num_round" -> 100)
var cachedDictDF: DataFrame = _

labelColName: String = delinquency_12
categaryCols: List[(String, org.apache.spark.sql.types.FloatType.type)] = List((orig_channel,FloatType), (first_home_buyer,FloatType), (loan_purpose,FloatType), (property_type,FloatType), (occupancy_status,FloatType), (property_state,FloatType), (product_type,FloatType), (relocation_mortgage_indicator,FloatType), (seller_name,FloatType), (mod_flag,FloatType))
numericCols: List[(String, org.apache.spark.sql.types.NumericType with Product with Serializable)] = List((orig_interest_rate,FloatType), (orig_upb,IntegerType), (orig_loan_term,IntegerType), (orig_ltv,FloatType), (orig_cltv,FloatType), (num_borrowers,FloatType), (dti,FloatType), (borrower_credit_score,FloatType), (num_units,IntegerType), (zip,IntegerType), (mortgage_insurance_percent,FloatType...


* Define Casting Process
This part is casting String column to Numbric. 
Example:
```
col_1
 "a"
 "b"
 "c"
 "a"
# After String ====> Numberic
col_1
 0
 1
 2
 0
```  
<br>

* Define function to get column dictionary

    Example
    ```
    col1 = [row(data="a",id=0), row(data="b",id=1)]
    ```

In [39]:
def genDictionary(etlDF: DataFrame, colNames: Seq[String]): DataFrame = {
    val cntTable = etlDF
      .select(posexplode(array(colNames.map(col(_)): _*)))
      .withColumnRenamed("pos", "column_id")
      .withColumnRenamed("col", "data")
      .filter("data is not null")
      .groupBy("column_id", "data")
      .count()
    val windowed = Window.partitionBy("column_id").orderBy(desc("count"))
    cntTable
      .withColumn("id", row_number().over(windowed))
      .drop("count")
  }

genDictionary: (etlDF: org.apache.spark.sql.DataFrame, colNames: Seq[String])org.apache.spark.sql.DataFrame


* Define function to convert string columns to numeric

In [40]:
def castStringColumnsToNumeric(inputDF: DataFrame, spark: SparkSession): DataFrame = {
    val cateColNames = categaryCols.map(_._1)
    cachedDictDF = genDictionary(inputDF, cateColNames).cache()

    // Generate the final table with all columns being numeric.
    cateColNames.foldLeft(inputDF) {
      case (df, colName) =>
        val colPos = cateColNames.indexOf(colName)
        val colDictDF = cachedDictDF
          .filter(col("column_id") === colPos)
          .drop("column_id")
          .withColumnRenamed("data", colName)
        df.join(broadcast(colDictDF), Seq(colName), "left")
          .drop(colName)
          .withColumnRenamed("id", colName)
    }
  }

castStringColumnsToNumeric: (inputDF: org.apache.spark.sql.DataFrame, spark: org.apache.spark.sql.SparkSession)org.apache.spark.sql.DataFrame


* Build the spark session and data reader

In [41]:
// Build the spark session and data reader as usual
val sparkSession = SparkSession.builder.appName("mortgage-gpu").getOrCreate
val reader = spark.read.option("header", false)
                              .option("nullValue", "")
                              .option("delimiter", "|")
                              .option("parserLib", "univocity")

sparkSession: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@2777350c
reader: org.apache.spark.sql.DataFrameReader = org.apache.spark.sql.DataFrameReader@1545dee


* Read CSV Files

In [42]:
val rawSet = reader
      .schema(rawSchema)
      .csv(rawPath)
rawSet.createOrReplaceTempView("raw")

rawSet: org.apache.spark.sql.DataFrame = [reference_pool_id: string, loan_id: bigint ... 106 more fields]


In [43]:
val perfSet = spark.sql("""
select loan_id, 
       date_format(to_date(monthly_reporting_period,"MMyyyy"), "MM/dd/yyyy") as monthly_reporting_period,
       upper(servicer) as servicer,
       interest_rate,
       current_actual_upb,
       loan_age,
       remaining_months_to_legal_maturity,
       adj_remaining_months_to_maturity,
       date_format(to_date(maturity_date,"MMyyyy"), "MM/yyyy") as maturity_date,
       msa,
       current_loan_delinquency_status,
       mod_flag,
       zero_balance_code,
       date_format(to_date(zero_balance_effective_date,"MMyyyy"), "MM/yyyy") as zero_balance_effective_date,
       date_format(to_date(last_paid_installment_date,"MMyyyy"), "MM/dd/yyyy") as last_paid_installment_date,
       date_format(to_date(foreclosed_after,"MMyyyy"), "MM/dd/yyyy") as foreclosed_after,
       date_format(to_date(disposition_date,"MMyyyy"), "MM/dd/yyyy") as disposition_date,
       foreclosure_costs,
       prop_preservation_and_repair_costs,
       asset_recovery_costs,
       misc_holding_expenses,
       holding_taxes,
       net_sale_proceeds,
       credit_enhancement_proceeds,
       repurchase_make_whole_proceeds,
       other_foreclosure_proceeds,
       non_interest_bearing_upb,
       principal_forgiveness_upb,
       repurchase_make_whole_proceeds_flag,
       foreclosure_principal_write_off_amount,
       servicing_activity_indicator,
       substring_index(substring_index(input_file_name(),'/',-1),'.',1) as quarter
from raw
""")

perfSet: org.apache.spark.sql.DataFrame = [loan_id: bigint, monthly_reporting_period: string ... 30 more fields]


In [44]:
val acqSet = spark.sql("""
select loan_id,
       orig_channel,
       seller_name,
       orig_interest_rate,
       orig_upb,
       orig_loan_term,
       date_format(to_date(orig_date,"MMyyyy"), "MM/yyyy") as orig_date,
       date_format(to_date(first_pay_date,"MMyyyy"), "MM/yyyy") as first_pay_date,
       orig_ltv,
       orig_cltv,
       num_borrowers,
       dti,
       borrower_credit_score,
       first_home_buyer,
       loan_purpose,
       property_type,
       num_units,
       occupancy_status,
       property_state,
       zip,
       mortgage_insurance_percent,
       product_type,
       coborrow_credit_score,
       mortgage_insurance_type,
       relocation_mortgage_indicator,
       substring_index(substring_index(input_file_name(),'/',-1),'.',1) as quarter
from raw
""").distinct

acqSet: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [loan_id: bigint, orig_channel: string ... 24 more fields]


* Define ETL Object

In [45]:
trait MortgageETL {
  var dataFrame: DataFrame = _

  def from(df: DataFrame): this.type = {
    dataFrame = df
    this
  }
}
val allCols = (categaryCols ++ numericCols).map(c => col(c._1))

object PerformanceETL extends MortgageETL {

  def prepare: this.type = {
    dataFrame = dataFrame
      .withColumn("monthly_reporting_period", to_date(col("monthly_reporting_period"), "MM/dd/yyyy"))
      .withColumn("monthly_reporting_period_month", month(col("monthly_reporting_period")))
      .withColumn("monthly_reporting_period_year", year(col("monthly_reporting_period")))
      .withColumn("monthly_reporting_period_day", dayofmonth(col("monthly_reporting_period")))
      .withColumn("last_paid_installment_date", to_date(col("last_paid_installment_date"), "MM/dd/yyyy"))
      .withColumn("foreclosed_after", to_date(col("foreclosed_after"), "MM/dd/yyyy"))
      .withColumn("disposition_date", to_date(col("disposition_date"), "MM/dd/yyyy"))
      .withColumn("maturity_date", to_date(col("maturity_date"), "MM/yyyy"))
      .withColumn("zero_balance_effective_date", to_date(col("zero_balance_effective_date"), "MM/yyyy"))
      .withColumn("current_actual_upb", col("current_actual_upb"))
      .withColumn("current_loan_delinquency_status", col("current_loan_delinquency_status"))
    this
  }

  def createDelinquency(spark: SparkSession): this.type = {
    val aggDF = dataFrame
      .select(
        col("quarter"),
        col("loan_id"),
        col("current_loan_delinquency_status"),
        when(col("current_loan_delinquency_status") >= 1, col("monthly_reporting_period")).alias("delinquency_30"),
        when(col("current_loan_delinquency_status") >= 3, col("monthly_reporting_period")).alias("delinquency_90"),
        when(col("current_loan_delinquency_status") >= 6, col("monthly_reporting_period")).alias("delinquency_180")
      )
      .groupBy("quarter", "loan_id")
      .agg(
        max("current_loan_delinquency_status").alias("delinquency_12"),
        min("delinquency_30").alias("delinquency_30"),
        min("delinquency_90").alias("delinquency_90"),
        min("delinquency_180").alias("delinquency_180")
      )
      .select(
        col("quarter"),
        col("loan_id"),
        (col("delinquency_12") >= 1).alias("ever_30"),
        (col("delinquency_12") >= 3).alias("ever_90"),
        (col("delinquency_12") >= 6).alias("ever_180"),
        col("delinquency_30"),
        col("delinquency_90"),
        col("delinquency_180")
      )

    val joinedDf = dataFrame
      .withColumnRenamed("monthly_reporting_period", "timestamp")
      .withColumnRenamed("monthly_reporting_period_month", "timestamp_month")
      .withColumnRenamed("monthly_reporting_period_year", "timestamp_year")
      .withColumnRenamed("current_loan_delinquency_status", "delinquency_12")
      .withColumnRenamed("current_actual_upb", "upb_12")
      .select("quarter", "loan_id", "timestamp", "delinquency_12", "upb_12", "timestamp_month", "timestamp_year")
      .join(aggDF, Seq("loan_id", "quarter"), "left_outer")

    // calculate the 12 month delinquency and upb values
    val months = 12
    val monthArray = 0.until(months).toArray
    val testDf = joinedDf
      // explode on a small amount of data is actually slightly more efficient than a cross join
      .withColumn("month_y", explode(lit(monthArray)))
      .select(
        col("quarter"),
        floor(((col("timestamp_year") * 12 + col("timestamp_month")) - 24000) / months).alias("josh_mody"),
        floor(((col("timestamp_year") * 12 + col("timestamp_month")) - 24000 - col("month_y")) / months).alias("josh_mody_n"),
        col("ever_30"),
        col("ever_90"),
        col("ever_180"),
        col("delinquency_30"),
        col("delinquency_90"),
        col("delinquency_180"),
        col("loan_id"),
        col("month_y"),
        col("delinquency_12"),
        col("upb_12")
      )
      .groupBy("quarter", "loan_id", "josh_mody_n", "ever_30", "ever_90", "ever_180", "delinquency_30", "delinquency_90", "delinquency_180", "month_y")
      .agg(max("delinquency_12").alias("delinquency_12"), min("upb_12").alias("upb_12"))
      .withColumn("timestamp_year", floor((lit(24000) + (col("josh_mody_n") * lit(months)) + (col("month_y") - 1)) / lit(12)))
      .withColumn("timestamp_month_tmp", pmod(lit(24000) + (col("josh_mody_n") * lit(months)) + col("month_y"), lit(12)))
      .withColumn("timestamp_month", when(col("timestamp_month_tmp") === lit(0), lit(12)).otherwise(col("timestamp_month_tmp")))
      .withColumn("delinquency_12", ((col("delinquency_12") > 3).cast("int") + (col("upb_12") === 0).cast("int")).alias("delinquency_12"))
      .drop("timestamp_month_tmp", "josh_mody_n", "month_y")

    dataFrame = dataFrame
      .withColumnRenamed("monthly_reporting_period_month", "timestamp_month")
      .withColumnRenamed("monthly_reporting_period_year", "timestamp_year")
      .join(testDf, Seq("quarter", "loan_id", "timestamp_year", "timestamp_month"), "left").drop("timestamp_year", "timestamp_month")
    this
  }
}

object AcquisitionETL extends MortgageETL {

  def createAcquisition(spark: SparkSession): this.type = {
    val nameMapping = NameMapping(spark, "from_seller_name", "to_seller_name")
    dataFrame = dataFrame
      .join(nameMapping, col("seller_name") === col("from_seller_name"), "left")
      .drop("from_seller_name")
      /* backup the original name before we replace it */
      .withColumn("old_name", col("seller_name"))
      /* replace seller_name with the new version if we found one in the mapping, or the old version
       if we didn't */
      .withColumn("seller_name", coalesce(col("to_seller_name"), col("seller_name")))
      .drop("to_seller_name")
      .withColumn("orig_date", to_date(col("orig_date"), "MM/yyyy"))
      .withColumn("first_pay_date", to_date(col("first_pay_date"), "MM/yyyy"))
    this
  }

  def cleanPrime(perfDF: DataFrame): this.type = {
    dataFrame = perfDF.join(dataFrame, Seq("loan_id", "quarter"), "inner").drop("quarter")
    this
  }
}


defined trait MortgageETL
allCols: List[org.apache.spark.sql.Column] = List(orig_channel, first_home_buyer, loan_purpose, property_type, occupancy_status, property_state, product_type, relocation_mortgage_indicator, seller_name, mod_flag, orig_interest_rate, orig_upb, orig_loan_term, orig_ltv, orig_cltv, num_borrowers, dti, borrower_credit_score, num_units, zip, mortgage_insurance_percent, current_loan_delinquency_status, current_actual_upb, interest_rate, loan_age, msa, non_interest_bearing_upb, delinquency_12)
defined object PerformanceETL
defined object AcquisitionETL


In [46]:
def transform(perfDF: DataFrame, acqDF: DataFrame, spark: SparkSession): DataFrame = {
    val etlPerfDF = PerformanceETL.from(perfDF)
      .prepare
      .createDelinquency(spark)
      .dataFrame
    val cleanDF = AcquisitionETL.from(acqDF)
      .createAcquisition(spark)
      .cleanPrime(etlPerfDF)
      .dataFrame

    // Convert to xgb required Dataset
    castStringColumnsToNumeric(cleanDF, spark)
      .select(allCols: _*)
      .withColumn(labelColName, when(col(labelColName) > 0, 1).otherwise(0))
      .na.fill(0.0f)
  }

transform: (perfDF: org.apache.spark.sql.DataFrame, acqDF: org.apache.spark.sql.DataFrame, spark: org.apache.spark.sql.SparkSession)org.apache.spark.sql.DataFrame


## Run ETL Process and Save the Result

In [47]:
val t0 = System.currentTimeMillis
val rawDF = transform(
      perfSet,
      acqSet,
      sparkSession
    )
rawDF.write.mode("overwrite").parquet(new Path(outPath, "data").toString)
val t1 = System.currentTimeMillis
println("Elapsed time : " + ((t1 - t0).toFloat / 1000) + "s")
sparkSession.stop()

org.apache.spark.SparkException:  Job aborted.